In [12]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import hardware

context.version = "1.0" # required during transition
context.choose_site(default="CHI@UC")
context.choose_project()
username = os.getenv('USER') # all exp resources will have this prefix

In [13]:
node_type = "gpu_rtx_6000"

In [14]:
gpu_start_times = [n.next_free_timeslot()[0] for n in hardware.get_nodes(node_type=node_type)]
current_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(minutes=1)
if min(gpu_start_times) > current_time:
    lease_start = min(gpu_start_times)
    print(f"There is no {node_type} available now, you can request one starting at {str(lease_start)} (UTC).")
else:
    lease_start = current_time
    print(f"A {node_type} IS available now, your lease will start right away.")


A gpu_rtx_6000 IS available now, your lease will start right away.


In [15]:
l = lease.Lease(f"repro-{username}-{node_type}", duration=datetime.timedelta(hours=3),
                start_date = lease_start  )
l.add_node_reservation(node_type = node_type, amount = 1) 
l.add_fip_reservation(1) 
l.submit(idempotent=True)

Waiting for lease to start...


Lease repro-ezrajoanerinata_gmail_com-gpu_rtx_6000 has reached status active


In [16]:
# continue here, whether using a lease created just now or one created earlier
l = lease.get_lease(f"repro-{username}-{node_type}")

In [ ]:
s = server.Server(
    f"colab-{username}-{node_type}", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

Waiting for server colab-ezrajoanerinata_gmail_com-gpu_rtx_6000's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


In [ ]:
reserved_fip = l.get_reserved_floating_ips()[0]
s.associate_floating_ip(reserved_fip)

In [ ]:
s.check_connectivity(host=reserved_fip)

In [21]:
print("ssh cc@" + reserved_fip)

ssh cc@192.5.86.225


Do the terminal first

In [22]:
from chi import ssh
node = ssh.Remote(reserved_fip) 

In [23]:
node.run("curl -sSL https://get.docker.com/ | sudo sh")
node.run("sudo groupadd -f docker; sudo usermod -aG docker $USER")

/opt/conda/lib/python3.12/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.225: b'ba9775cf9eb14619f8a1005b2c5bd046'
  warnings.warn(


# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c doc

Client: Docker Engine - Community
 Version:           28.2.2
 API version:       1.50
 Go version:        go1.24.3
 Git commit:        e6534b4
 Built:             Fri May 30 12:07:27 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.2.2
  API version:      1.50 (minimum version 1.24)
  Go version:       go1.24.3
  Git commit:       45873be
  Built:            Fri May 30 12:07:27 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

In [24]:
node = ssh.Remote(reserved_fip) # note: need a new SSH session to get new group membership
node.run("docker run hello-world")

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
e6590344b1a5: Pulling fs layer
e6590344b1a5: Verifying Checksum
e6590344b1a5: Download complete
e6590344b1a5: Pull complete
Digest: sha256:0b6a027b5cf322f09f6706c754e086a232ec1ddba835c8a15c6cb74ef0d43c29
Status: Downloaded newer image for hello-world:latest



Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



<Result cmd='docker run hello-world' exited=0>

In [25]:
# get NVIDIA container toolkit 
node.run("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
node.run("sudo apt update")
node.run("sudo apt-get install -y nvidia-container-toolkit")
node.run("sudo nvidia-ctk runtime configure --runtime=docker")
node.run("sudo systemctl restart docker")

deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/stable/deb/$(ARCH) /
#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/experimental/deb/$(ARCH) /


Hit:1 https://download.docker.com/linux/ubuntu noble InRelease
Get:2 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease [1477 B]
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:8 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  Packages [20.4 kB]
Fetched 21.9 kB in 1s (17.8 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
43 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libnvidia-container-tools libnvidia-container1 nvidia-containe

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 5933 kB in 1s (4269 kB/s)
Selecting previously unselected package libnvidia-container1:amd64.
(Reading database ... 113812 files and directories currently installed.)
Preparing to unpack .../libnvidia-container1_1.17.7-1_amd64.deb ...
Unpacking libnvidia-container1:amd64 (1.17.7-1) ...
Selecting previously unselected package libnvidia-container-tools.
Preparing to unpack .../libnvidia-container-tools_1.17.7-1_amd64.deb ...
Unpacking libnvidia-container-tools (1.17.7-1) ...
Selecting previously unselected package nvidia-container-toolkit-base.
Preparing to unpack .../nvidia-container-toolkit-base_1.17.7-1_amd64.deb ...
Unpacking nvidia-container-toolkit-base (1.17.7-1) ...
Selecting previously unselected package nvidia-container-toolkit.
Preparing to unpack .../nvidia-container-toolkit_1.17.7-1_amd64.deb ...
Unpacking nvidia-container-toolkit (1.17.7-1) ...
Setting up nvidia-container-toolkit-base (1.17.7-1) ...
Setting up libnvidia-container1:amd64 (1.17.7-1) ...
Setting up lib

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
time="2025-06-02T10:38:11Z" level=info msg="Config file does not exist; using empty config"
time="2025-06-02T10:38:11Z" level=info msg="Wrote updated config to /etc/docker/daemon.json"
time="2025-06-02T10:38:11Z" level=info msg="It is recommended that docker daemon be restarted."


<Result cmd='sudo systemctl restart docker' exited=0>

In [26]:
# check that we can see GPU from inside container
node.run("docker run --rm --gpus all ubuntu nvidia-smi")

Unable to find image 'ubuntu:latest' locally
latest: Pulling from library/ubuntu
0622fac788ed: Pulling fs layer
0622fac788ed: Verifying Checksum
0622fac788ed: Download complete
0622fac788ed: Pull complete
Digest: sha256:6015f66923d7afbc53558d7ccffd325d43b4e249f41a6e93eef074c9505d2233
Status: Downloaded newer image for ubuntu:latest


Mon Jun  2 10:38:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 6000                Off |   00000000:3B:00.0 Off |                  Off |
| 33%   20C    P8             16W /  260W |       2MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

<Result cmd='docker run --rm --gpus all ubuntu nvidia-smi' exited=0>

In [27]:
node.run("docker run -d -p 8888:8888 --rm --gpus all quay.io/jupyter/pytorch-notebook:cuda12-latest")

Unable to find image 'quay.io/jupyter/pytorch-notebook:cuda12-latest' locally
cuda12-latest: Pulling from jupyter/pytorch-notebook
f03f49e66a78: Pulling fs layer
06ac3029a007: Pulling fs layer
ed2da93444d8: Pulling fs layer
4f4fb700ef54: Pulling fs layer
10bf4b5ce5bd: Pulling fs layer
02d0c2faec92: Pulling fs layer
eac6e0bb5c57: Pulling fs layer
0ea87f6d444e: Pulling fs layer
840146cd6907: Pulling fs layer
f487082da68d: Pulling fs layer
2929654834f8: Pulling fs layer
7b59d9faa57a: Pulling fs layer
dd40919da564: Pulling fs layer
af3290429610: Pulling fs layer
02d0c2faec92: Waiting
eac6e0bb5c57: Waiting
1bcaa076a606: Pulling fs layer
9abcb396bde3: Pulling fs layer
10bf4b5ce5bd: Waiting
3cbc019f5cd1: Pulling fs layer
4f4fb700ef54: Waiting
1b27d7959a26: Pulling fs layer
0ea87f6d444e: Waiting
595de7212bee: Pulling fs layer
2929654834f8: Waiting
f487082da68d: Waiting
bf6139a8c23e: Pulling fs layer
7b59d9faa57a: Waiting
1bcaa076a606: Waiting
67f0164cae0e: Pulling fs layer
af3290429610: Waitin

1eeb2be00c9bec7580e5ebf06af3ac49d4ad32270e9bdd55e8ba099d6f4b503b


<Result cmd='docker run -d -p 8888:8888 --rm --gpus all quay.io/jupyter/pytorch-notebook:cuda12-latest' exited=0>

In [28]:
node.run("docker ps -q | xargs -L 1 docker logs")

Entered start.sh with args: start-notebook.py


Executing: jupyter lab


Running hooks in: /usr/local/bin/start-notebook.d as uid: 1000 gid: 100
Done running hooks in: /usr/local/bin/start-notebook.d
Running hooks in: /usr/local/bin/before-notebook.d as uid: 1000 gid: 100
Sourcing shell script: /usr/local/bin/before-notebook.d/10activate-conda-env.sh
Done running hooks in: /usr/local/bin/before-notebook.d
Executing the command: start-notebook.py
[I 2025-06-02 10:46:55.258 ServerApp] jupyter_lsp | extension was successfully linked.
[I 2025-06-02 10:46:55.260 ServerApp] jupyter_server_mathjax | extension was successfully linked.
[I 2025-06-02 10:46:55.263 ServerApp] jupyter_server_terminals | extension was successfully linked.
[I 2025-06-02 10:46:55.266 ServerApp] jupyterlab | extension was successfully linked.
[I 2025-06-02 10:46:55.266 ServerApp] jupyterlab_git | extension was successfully linked.
[I 2025-06-02 10:46:55.269 ServerApp] nbclassic | extension was successfully linked.
[I 2025-06-02 10:46:55.269 ServerApp] nbdime | extension was successfully lin

<Result cmd='docker ps -q | xargs -L 1 docker logs' exited=0>

^^^ Check Ip ^^^

Run below if done with anything

In [10]:
# setup environment - if you made any changes in the top part, make the same changes here
import chi, os
from chi import lease, magic, context

context.version = "1.0" # required during transition

context.choose_site(default="CHI@UC")
context.choose_project()
username = os.getenv('USER') # all exp resources will have this prefix

node_type = "gpu_rtx_6000"
l = lease.get_lease(f"repro-{username}-{node_type}")

In [11]:
# un-comment to free resources
chi.magic.cleanup_resources(l.id)


Deleted lease 2d10ed55-a13e-4f0a-bc7a-301c9d0e87d5
